In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import  torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from tqdm import tqdm  # For nice progress bar!

# CNN

In [ ]:
class CNN(nn.Module):
  def __init__(self, in_channels=1, num_classes=10):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(
        in_channels=1,
        out_channels=8,
        kernel_size=3 ,
        stride=1,
        padding=1)  # it called same convolution
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv2 = nn.Conv2d(
        in_channels=8,
        out_channels=16,
        kernel_size=3,
        stride=1,
        padding=1)
    self.fc1 = nn.Linear(16 * 7 * 7, num_classes)


  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc1(x)

    return x


In [ ]:
model = CNN()
x = torch.randn(64, 1, 28, 28)
model(x).shape


torch.Size([64, 10])

In [ ]:
device= torch.device('cuda' if torch.cuda.is_available else 'cpu')
device

device(type='cuda')

In [ ]:
in_channels = 1
num_classes = 10
learning_rate= 0.001
batch_size = 64
num_epochs = 5

In [ ]:
train_dataset = datasets.MNIST(root='datasets/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='datasets/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
model = CNN(in_channels= in_channels, num_classes = num_classes).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
for epoch in range(num_epochs):
  for data_idx, (data, target) in enumerate(tqdm(train_loader)):

    # get data to cuda if possible
    data = data.to(device = device)
    target = target.to(device = device)

    # forward
    scores = model(data)
    loss = criterion(scores, target)

    #backward
    optimizer.zero_grad() # Essentially set all the gradients to zero for each batch
    loss.backward()

    # gradient descent or adam step
    optimizer.step()

100%|██████████| 938/938 [00:09<00:00, 99.48it/s] 


In [ ]:
def check_accuracy(loader, model):

  if loader.dataset.train:
    print('Checking Accuracy of train data: ')
  else:
    print('Checking Accuracy of test data: ')

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    # We don't have to compute the gradient that would be unnessessory computing
    for x, y in loader:
      x = x.to(device= device)
      y = y.to(device = device)


      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")
  model.train()


In [ ]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)


Checking Accuracy of train data: 
Got 59552 / 60000 with accuracy 99.25
Checking Accuracy of test data: 
Got 9866 / 10000 with accuracy 98.66
